# Co-Occurrence und KWIC

Wie du im letzten Teil gesehen hast, konnten wir relativ einfach Wordclouds generieren und anzeigen. Diese zeigen dir, welche Themen im Heise Newsticker behandelt werden.

Allerdings fehlen dir zur genaueren Interpretation noch *Kontextinformationen*. Die wirst du nun ermitteln

## Daten einladen

Du beginnst zunächst wieder mit dem Laden der linugistisch analysierten Daten aus der Datenbank. Das wird auch in Zukunft häufig dein erster Arbeitsschritt sein. Wenn du möchtest, kannst du auch nur eine Teilmenge mit `LIMIT` auswählen:

In [ ]:
!pip install textacy

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles ORDER BY datePublished", sql, index_col="id")

## Vokabular bestimmen

Im ersten Schritt bestimmst du nun das Vokabular, das du analysieren möchtest. Dazu benötigst du wieder die Funktion, die die Felder in die Tokens zerlegt:

In [ ]:
import regex as re
def single_words(df, field):
    return [w for words in df[field] for w in re.split(r'\||\#', words)]

Häufig ist das `nav`-Feld, das Substantive, Adjektive und Verben enthält, besonders gut für die Analyse geeignet. Mithilfe eines `Counter` bestimmst du die häufigsten Wörter in dem Feld, lässt aber die Stopwords weg. Die 10.000 häufigsten Wörter speicherst du in einer separaten Variable `voc` ab:

In [ ]:
from collections import Counter
from spacy.lang.de.stop_words import STOP_WORDS

nav = Counter(single_words(df, "nav"))
for w in STOP_WORDS:
    nav[w] = 0
    
voc = [w[0] for w in nav.most_common(10000)]

In einem Fenster von fünf Wörtern werden nun die sog. *Co-Occurrences* ermittelt. Das sind die Wörter, die dort gemeinsam auftreten. Auch hier werden die Stopwords ausgelassen. Das Fenster wird vom "mittleren" Wort halb nach vorne und halb nach hinten gerechnet. Die Konstruktion mit dem `defaultdict` und dem `Counter` erspart dir viele `if`-Abfragen:

In [ ]:
from collections import Counter, defaultdict
from tqdm import tqdm
import regex as re

# defaultdict liefert das Argument zurück, wenn key noch unbekannt
# in diesem Fall also einen neuen (leeren) Counter
c = defaultdict(lambda: Counter())

window = 5 # sollte ungerade sein
skip = (window - 1) // 2
for doc in tqdm(df["nav"]):
    # Stopwords eliminieren
    tokens = [w for w in re.split(r'\||\#', doc) if w not in STOP_WORDS]
    for i, w in enumerate(tokens):
        if w in voc:
            for j in range(max(0, i-skip), i):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1
            for j in range(i+1, min(i+1+skip, len(tokens))):
                if tokens[j] in voc:
                    c[w][tokens[j]] += 1

Nun kannst du ermitteln, welche Begriffe besonders häufig zusammen mit `Apple` vorkommen:

In [ ]:
c["Apple"].most_common()

Das sieht schon sehr gut aus. Teste es noch mit `Google`:

In [ ]:
c["Google"].most_common()

Auch dieses Ergebnis ist richtig gut und wie erwartet. Die Google-Dienste erscheinen eigentlich alle und du kannst damit feststellen, dass die Meldungen nicht nur die richtigen Wörter enthalten, sondern auch die richtigen Themen besprechen.

## Keywords in Context

Jetzt weißt du aber noch nicht, wie die Wörter wirklich verwendet werden. Dazu kann Keyword-in-Context [KWIC](https://de.wikipedia.org/wiki/Permutiertes_Register) helfen. Dazu werden für die Einzelmeldungen jeweils immer ein bestimmtes Festner von Buchstaben angezeigt, die um das gewählte Wort vorkommen. Die Funktion ist einfach implementiert, ein *regulärer Ausdruck* wird dazu dynamisch konstruiert:

In [ ]:
import regex as re
def kwic(word, texts, window_size):
    res = []
    for text in texts:
        context = (window_size//2)*'.'
        kwic = context + r'\b' + word + r'\b' + context
        match = re.findall(kwic, text)
        for m in match:
            res.append(m)
            
    return res

Die Funktion erfordert ein Array von Strings, das kannst du aus dem `DataFrame` einfach ermitteln:

In [ ]:
text = df["text"].map(str).values

Der Aufruf kann einen Augenblick dauern, weil es viele Meldungen mit `Apple` gibt:

In [ ]:
kwic("Apple", text, 60)

Jetzt kannst du die Einzelmeldungen sehen. Alle zu lesen, ist natürlich etwas aufwändiger, als die Co-Occurrence oben auszuwerten.

## *Context is King*

Um Texte genauer zu verstehen, sind Kontextinformationen eminent wichtig. Mit *Co-Occurrence* und *KWIC* hast du zwei Methoden kennengelernt, mit denen du den Kontext einfach bestimmen kannst.